<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/low_level/oss_ingestion_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building RAG from Scratch (Open-source only!)

In this tutorial, we show you how to build a data ingestion pipeline into a vector database, and then build a retrieval pipeline from that vector database, from scratch.

Notably, we use a fully open-source stack:

- Sentence Transformers as the embedding model
- Postgres as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [1]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: nltk
    Found existing installation

In [2]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Llama CPP

In this notebook, we use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [3]:
!pip install llama-cpp-python

In [4]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


total size (MB): 7365.83


7025it [00:49, 142.92it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_load

In [29]:
llm

LlamaCPP(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7b163b200370>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7b1736b0da20>, completion_to_prompt=<function default_completion_to_prompt at 0x7b1736a1d6c0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model_url='https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf', model_path='/tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf', temperature=0.1, max_new_tokens=256, context_window=3900, generate_kwargs={'temperature': 0.1, 'max_tokens': 256}, model_kwargs={'n_ctx': 3900, 'verbose': True, 'n_gpu_layers': 1}, verbose=True)

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/module_guides/storing/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [20]:
!pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

In [22]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "jerry"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (::1), port 5432 failed: Cannot assign requested address
	Is the server running on that host and accepting TCP/IP connections?


In [21]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

NameError: name 'db_name' is not defined

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

### 1. Load Data

In [5]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

--2024-09-22 14:00:31--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2024-09-22 14:00:31--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  83.1MB/s    in 0.2s    

2024-09-22 14:00:32 (83.1 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [6]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [7]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [8]:
documents

[Document(id_='4c09e432-4501-4e87-92b0-f472c73a0e41', embedding=None, metadata={'total_pages': 77, 'file_path': './data/llama2.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗\nLouis Martin†\nKevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan S

### 2. Use a Text Splitter to Split Documents

In [9]:
from llama_index.core.node_parser import SentenceSplitter

In [10]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [11]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [31]:
doc_idxs

[0,
 1,
 1,
 1,
 2,
 3,
 4,
 5,
 6,
 6,
 7,
 7,
 8,
 9,
 9,
 10,
 10,
 11,
 12,
 13,
 14,
 14,
 15,
 16,
 16,
 17,
 18,
 19,
 19,
 20,
 21,
 22,
 23,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 35,
 36,
 36,
 37,
 37,
 38,
 38,
 39,
 39,
 40,
 40,
 41,
 41,
 42,
 42,
 43,
 43,
 44,
 45,
 46,
 46,
 47,
 48,
 48,
 49,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 58,
 59,
 60,
 61,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 68,
 69,
 70,
 70,
 71,
 72,
 72,
 73,
 73,
 73,
 74,
 74,
 75,
 76]

In [32]:
text_chunks

['Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗\nLouis Martin†\nKevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi\nAlan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojn

### 3. Manually Construct Nodes from Text Chunks

In [13]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [33]:
nodes

[TextNode(id_='4c95807e-a914-4703-8f0b-d7dcdafda8ce', embedding=[-0.04617450758814812, 0.03491324186325073, -0.013553637079894543, -0.007586942054331303, 0.017399096861481667, -0.00029551811167038977, -0.00876539945602417, 0.015330236405134201, 0.025656530633568764, -0.006066991947591305, -0.010280652903020382, -0.041364967823028564, 0.059217993170022964, 0.018592797219753265, 0.05700131133198738, 0.03878618776798248, 0.017730018123984337, -0.016568612307310104, 0.006337070371955633, 0.03478853404521942, -0.012293384410440922, -0.044170137494802475, 0.014340607449412346, 0.006127842236310244, -0.02293403632938862, 0.020668381825089455, -0.035216327756643295, -0.03513773903250694, -0.007999951019883156, -0.22341391444206238, -0.005436125211417675, 0.006865988951176405, 0.0467403270304203, 0.008483080193400383, -0.05644073709845543, -0.008972191251814365, -0.0028833027463406324, -0.022347010672092438, -0.02375669777393341, 0.017667457461357117, 0.006298318039625883, 0.0019439550815150142

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [14]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [34]:
node_embedding

[-0.047984685748815536,
 -0.011453934945166111,
 -0.007028207182884216,
 0.0034323756117373705,
 0.02276965230703354,
 0.018554989248514175,
 -0.014038130640983582,
 0.002151703927665949,
 0.026390187442302704,
 -0.0009579498437233269,
 0.025721773505210876,
 -0.05930526927113533,
 0.044593654572963715,
 0.030027762055397034,
 0.02441609837114811,
 0.014880215749144554,
 -0.0026261017192155123,
 -0.028119193390011787,
 0.01790904626250267,
 0.01979658752679825,
 0.006029593292623758,
 0.008445304818451405,
 0.002280022483319044,
 -0.017668798565864563,
 -0.007935324683785439,
 0.01918702945113182,
 -0.04521893337368965,
 -0.01904577761888504,
 -0.02033078297972679,
 -0.26256081461906433,
 -0.0033913792576640844,
 0.0003759673563763499,
 0.031121263280510902,
 0.02606041543185711,
 -0.07592485100030899,
 -0.0015833477955311537,
 -0.018036983907222748,
 -0.02042837254703045,
 -0.01974891684949398,
 0.024013115093111992,
 -0.018281592056155205,
 0.017016790807247162,
 0.03322617709636688,

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

In [15]:
vector_store.add(nodes)

NameError: name 'vector_store' is not defined

In [35]:
nodes

[TextNode(id_='4c95807e-a914-4703-8f0b-d7dcdafda8ce', embedding=[-0.04617450758814812, 0.03491324186325073, -0.013553637079894543, -0.007586942054331303, 0.017399096861481667, -0.00029551811167038977, -0.00876539945602417, 0.015330236405134201, 0.025656530633568764, -0.006066991947591305, -0.010280652903020382, -0.041364967823028564, 0.059217993170022964, 0.018592797219753265, 0.05700131133198738, 0.03878618776798248, 0.017730018123984337, -0.016568612307310104, 0.006337070371955633, 0.03478853404521942, -0.012293384410440922, -0.044170137494802475, 0.014340607449412346, 0.006127842236310244, -0.02293403632938862, 0.020668381825089455, -0.035216327756643295, -0.03513773903250694, -0.007999951019883156, -0.22341391444206238, -0.005436125211417675, 0.006865988951176405, 0.0467403270304203, 0.008483080193400383, -0.05644073709845543, -0.008972191251814365, -0.0028833027463406324, -0.022347010672092438, -0.02375669777393341, 0.017667457461357117, 0.006298318039625883, 0.0019439550815150142

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [16]:
query_str = "Can you tell me about the key concepts for safety finetuning"

### 1. Generate a Query Embedding

In [17]:
query_embedding = embed_model.get_query_embedding(query_str)

### 2. Query the Vector Database

In [36]:
query_embedding

[-0.05745978280901909,
 0.0006168354884721339,
 0.007399786729365587,
 -0.02610776200890541,
 0.010598313063383102,
 0.007987488061189651,
 0.06677503138780594,
 0.03960992023348808,
 0.002579606371000409,
 -0.004290149547159672,
 -0.051723413169384,
 -0.03663443401455879,
 0.01641760766506195,
 0.02025303989648819,
 0.005108687561005354,
 0.003774319775402546,
 0.03296408802270889,
 0.032009307295084,
 -0.00843996275216341,
 0.01931263506412506,
 -0.0045065805315971375,
 0.031248852610588074,
 -0.02401266060769558,
 -0.020971687510609627,
 0.0049607721157372,
 -0.0026956431102007627,
 0.01749047264456749,
 0.0022530758287757635,
 -0.01721314713358879,
 -0.19273662567138672,
 -0.046928491443395615,
 -0.03517274558544159,
 -0.02127940207719803,
 -0.007309691980481148,
 -0.020944027230143547,
 -0.008686776272952557,
 -0.028816942125558853,
 0.0312635637819767,
 -0.004988432861864567,
 -0.003774962155148387,
 0.047360800206661224,
 0.017691636458039284,
 -0.008748915046453476,
 -0.0091419

In [18]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [37]:
vector_store_query

VectorStoreQuery(query_embedding=[-0.05745978280901909, 0.0006168354884721339, 0.007399786729365587, -0.02610776200890541, 0.010598313063383102, 0.007987488061189651, 0.06677503138780594, 0.03960992023348808, 0.002579606371000409, -0.004290149547159672, -0.051723413169384, -0.03663443401455879, 0.01641760766506195, 0.02025303989648819, 0.005108687561005354, 0.003774319775402546, 0.03296408802270889, 0.032009307295084, -0.00843996275216341, 0.01931263506412506, -0.0045065805315971375, 0.031248852610588074, -0.02401266060769558, -0.020971687510609627, 0.0049607721157372, -0.0026956431102007627, 0.01749047264456749, 0.0022530758287757635, -0.01721314713358879, -0.19273662567138672, -0.046928491443395615, -0.03517274558544159, -0.02127940207719803, -0.007309691980481148, -0.020944027230143547, -0.008686776272952557, -0.028816942125558853, 0.0312635637819767, -0.004988432861864567, -0.003774962155148387, 0.047360800206661224, 0.017691636458039284, -0.008748915046453476, -0.00914196949452161

In [23]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

NameError: name 'vector_store' is not defined

### 3. Parse Result into a Set of Nodes

In [24]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

NameError: name 'query_result' is not defined

### 4. Put into a Retriever

In [25]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [26]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

NameError: name 'vector_store' is not defined

## Plug this into our RetrieverQueryEngine to synthesize a response

In [27]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

NameError: name 'retriever' is not defined

In [28]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)

NameError: name 'query_engine' is not defined

In [ ]:
print(str(response))

 Based on the results shown in Table 3, Llama 2 outperforms all open-source models on most of the benchmarks, with an average improvement of around 5 points over the next best model (GPT-3.5).


In [ ]:
print(response.source_nodes[0].get_content())